In [8]:
from langchain.llms import HuggingFaceHub
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate
from langchain_huggingface import HuggingFaceEmbeddings
from langchain.document_loaders import JSONLoader
from langchain.retrievers.multi_query import MultiQueryRetriever
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain.vectorstores import Chroma

In [2]:
from dotenv import load_dotenv
load_dotenv()

True

In [3]:
from langchain_groq import ChatGroq
from langchain_core.messages import HumanMessage

model = ChatGroq(model="gemma2-9b-it")

In [4]:
prompt_template = """استخدم المعلومات التالية للإجابة عن السؤال باللغة العربية. إذا لم تتمكن من الإجابة استنادًا إلى المعلومات، فقط قل "لا أعرف".

معلومات إضافية: {context}

سؤال باللغة العربية: {question}

إجابة باللغة العربية:"""

PROMPT = PromptTemplate(
    template=prompt_template, input_variables=["context", "question"]
)

In [6]:
loader = JSONLoader(file_path='scraped_fatwas.json', jq_schema='.[].combined_text')
documents = loader.load()

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
splits = text_splitter.split_documents(documents)

In [10]:
embeddings = HuggingFaceEmbeddings(model_name="aubmindlab/bert-base-arabertv2")

No sentence-transformers model found with name aubmindlab/bert-base-arabertv2. Creating a new one with mean pooling.
/opt/miniconda3/envs/langchain/lib/python3.12/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [11]:
vectorstore = Chroma.from_documents(documents=splits, embedding=embeddings)

In [12]:
retriever = vectorstore.as_retriever(search_kwargs={"k": 3})
qa_chain = RetrievalQA.from_chain_type(
    model,
    retriever=retriever,
    return_source_documents=True,
    chain_type_kwargs={"prompt": PROMPT}
)

In [13]:
question = "حكم العمل في شركة تبيع بزيادة في الثمن المؤجل"
result = qa_chain({"query": question})

/var/folders/yl/33g63q2s16qg4334_f6pknm00000gn/T/ipykernel_1025/4244202644.py:2: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use invoke instead.
  result = qa_chain({"query": question})


In [14]:
print("الإجابة:")
print(result["result"])

print("\nالوثائق المصدر:")
for doc in result["source_documents"]:
    print(f"النص: {doc.page_content}")
    print("---")

الإجابة:
حُكم العمل في شركة تبيع بزيادة في الثمن المؤجل، إذا كان العمل في مثل هذه الشركة يتوافق مع الضوابط الشرعية في بيع التقسيط، فهو جائز.

وذلك بناءً على ما ورد في الفتوى رقم (53/2/6) الصادرة في 14 مارس 1990م، والتي أجازت الزيادة في الثمن المؤجل عن الثمن الحال. 


الوثائق المصدر:
النص: 14مارس 1990م، وذلك في قراره رقم (53/2/6) بشأن البيع بالتقسيط. وفيه:تجوز الزيادة في الثمن المؤجل عن الثمن الحال.... إلخ. انتهى. وعليه؛ فعمل المحاسب في مثل هذه الشركات التي تلتزم الضوابط الشرعية في بيع التقسيط جائز. والله أعلم. يمكنك البحث عن الفتوى من خلال البريد الإلكتروني جميع الحقوق محفوظة © 2023 - 1998 لشبكة إسلام ويب جميع الحقوق محفوظة © 2023 - 1998 لشبكة إسلام ويب
---
النص: وإن لم يثب عليها، يعني وإن لم يعوض عنها. انتهى. وإذا كنت أهديت له بسبب أنه سيتزوجك؛ فبعض أهل العلم يرى لك الرجوع في هبتك ما دام الزواج لم يتم، لكن الرجوع في الهبة يشترط فيه أن تكون الهبة باقية في ملك الموهوب لم تتغير، وراجعي الفتوى:127875. والله أعلم. يمكنك البحث عن الفتوى من خلال البريد الإلكتروني جميع الحقوق محفوظة © 2023 - 